In [15]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit

In [16]:
def src_Loc(rec_pos, t_rec,c,src_pos):
    '''The function returns the difference between the position of source(guessed) and receiver location as calculated from 
    measurement of time difference of arrival with the distance calculated from position of source and receiver. We aim to 
    minimise this quantity

    rec_pos = a matrix with position of receivers stored in the following format [[x_1, y_1],[x_2,y_2]]
    t_rec  = the time of arrival as measured from data
    source_loc = the guessed source location for which the function is being evaluated
    
    '''
    
    n_rec = np.shape(rec_pos)[0]
    t_rec_rel1 = np.zeros(shape=(n_rec-1,1))   # time difference calculated wrt the value in first receiver
    
    t_rec_rel1[:] = t_rec[1:] -t_rec[0]     
    
    rec_pos_src_rel = np.zeros(shape =(n_rec-1,2)) # Calculating the relative position of recivers wrt the source
    
    rec_pos_src_rel[:] = rec_pos[1:] -src_pos*np.ones_like(rec_pos_src_rel)  # relative position of receiver 1 wrt the source
    
    rec1_pos_src_rel = rec_pos[0] -src_pos # the position of receiver one wrt the source
    
    rec_dist_src_rel = np.zeros(shape=(n_rec-1,1))
    
    rec1_dist_src_rel = np.sqrt(rec1_pos_src_rel[0]**2 + rec1_pos_src_rel[1]**2)
    
    rec_dist_src_rel[:] = np.sqrt(rec_pos_src_rel[:,0]**2 + rec_pos_src_rel[:,1]**2).reshape(n_rec-1,1)
    
    delta_r = t_rec_rel1*c
    
    error = np.zeros(shape =(n_rec-1,1))   # defing the error to be minimized
    error[:] = delta_r[:] - (rec_dist_src_rel[:]-rec1_dist_src_rel)   
    
    error_to_min =np.dot(error.T,error) # Getting a single value instead of an array
    
    return error_to_min
        

In [17]:
# Testing out with 101 receivers placed in a line

rec_pos_act= np.zeros(shape =(9,2))
rec_pos_act_4= np.zeros(shape =(12,2)) # Let us say that there are 3 reeivers placed at origin
rec_pos_act[:,:] = 0
rec_pos_act_4[:,:] = 0# The deployer is at origin. 
noise_r = np.random.normal(loc=0.0,scale =100,size = (9,2))
noise_r_2=np.random.normal(loc=0.0,scale =100,size = (9,2))
noise_r_3=np.random.normal(loc=0.0,scale =100,size = (9,2))
noise_r_4=np.random.normal(loc=0.0,scale =100,size = (12,2))
t_rec_ideal = np.zeros(shape =(9,1))
t_rec_ideal_4 = np.zeros(shape =(12,1))# ideal value at receivers
rec_pos_mes= rec_pos_act +noise_r
rec_pos_mes_2= rec_pos_act +noise_r_2
rec_pos_mes_4= rec_pos_act_4 +noise_r_4
t_rec_ideal[0:9] = np.sqrt((rec_pos_mes[0:9,1]-400)**2 + (rec_pos_mes[0:9,0]-400)**2).reshape(9,1)
t_rec_ideal_4[0:12] = np.sqrt((rec_pos_mes_4[0:12,1]-400)**2 + (rec_pos_mes_4[0:12,0]-400)**2).reshape(12,1)
noise_t = np.random.normal(loc=0.0,scale =0.2,size = (9,1))
noise_t_4 = np.random.normal(loc=0.0,scale =0.2,size = (12,1))
t_Rec_real = t_rec_ideal + noise_t
t_Rec_real_4 = t_rec_ideal_4 + noise_t_4

c = 1 # velocity of propagation

# Note the different order in which the noises are added this time. 

In [18]:
noise_r_3=np.random.normal(loc=0.0,scale =100,size = (9,2))
noise_r_3

array([[ -28.65942936, -108.41655106],
       [ -28.08944596, -135.0094925 ],
       [  58.90642239,    2.16819162],
       [ 148.31690843, -151.69996762],
       [  68.59055796, -193.76050014],
       [  56.67969615,  -19.51351877],
       [  35.846772  , -148.79984104],
       [ -98.03690549,   85.4077279 ],
       [ 181.43237513, -128.63336706]])

In [ ]:
noise_r_3=np.random.normal(loc=0.0,scale =100,size = (9,2))
x,y =np.mgrid[-500:500:1000j,-500:500:1000j]  # a 2D grid for defining domain and evaluation of source position
error = np.zeros(shape=(1000,1000)) # To store the values at various points
error2 = np.zeros(shape=(1000,1000)) 
rec_pos_modified = np.zeros_like(rec_pos_act)
rec_pos_modified[:] = rec_pos_act[:] + noise_r_3[:]  # Can modify later if required
rec_pos_mes_2[:]=rec_pos_mes_2[:]+ noise_r_3[:]
spos1=[]
spos2=[]
spos=np.zeros(shape =(164,2))
t_avg =np.mean(t_Rec_real)
rad_domain = np.sqrt(x**2 + y**2) # radius from the origin of each point in domain
r_avg = t_avg*c
n=0
t=0
for m in range(8):
    for l in range (m,9):
        for k in range(l,9):
                    if(k!=m or k!=l):
                        for i in range(1000):
                             for j in range(1000):
                                if rad_domain[i,j] > r_avg  - 50  and  rad_domain[i,j] < r_avg  + 50:
                                    error[i,j] = src_Loc(rec_pos=np.array([rec_pos_modified[m],rec_pos_modified[l],rec_pos_modified[k]]),t_rec=np.array([t_Rec_real[m],t_Rec_real[l],t_Rec_real[k]]),c=c,src_pos=np.array([x[i,j],y[i,j]]))
                                else:
                                    error[i,j] = np.max(error)+1e8    
                        n+=1
                        t+=1
                        print(t)
                        spos[n-1,0]=x[np.where(error == np.min(error))][0]
                        spos[n-1,1]=y[np.where(error == np.min(error))][0]
                        plt.figure(figsize=(8,6))
                        plt.contourf(x,y,error,levels=50)
                        plt.plot(rec_pos_mes[:,0],rec_pos_mes[:,1],'b.',label='Receivers')
                        plt.plot(rec_pos_mes_2[:,0],rec_pos_mes_2[:,1],'g.',label='Receivers')
                        plt.plot(spos[:,0],spos[:,1],'r.',label='Source position estimated_2')
                        plt.plot(400,400,'r^',label='Source position Actual')
                        plt.plot(x[np.where(error == np.min(error))],y[np.where(error == np.min(error))],'g^',label='Source position estimated')
                        plt.plot()
                        plt.legend()
                        plt.xlim(-500,500)
                        plt.ylim(-500,500)
                        plt.xlabel('X(m)')
                        plt.ylabel('Y(m)')
                        plt.axis('equal')
                        plt.colorbar()
                        plt.title('Error contour for receivers with error in both time of arrival and known position')
                        spos[:,0]=spos[:,0]-400
                        spos[:,1]=spos[:,1]-400
              

In [ ]:
noise_r_4=np.random.normal(loc=0.0,scale =100,size = (12,2))
x,y =np.mgrid[-500:500:1000j,-500:500:1000j]  # a 2D grid for defining domain and evaluation of source position
error = np.zeros(shape=(1000,1000)) # To store the values at various points
error2 = np.zeros(shape=(1000,1000)) 
rec_pos_modified_4 = np.zeros_like(rec_pos_act_4)
rec_pos_modified_4[:] = rec_pos_act_4[:] + noise_r_3[:]  # Can modify later if required
rec_pos_mes_4[:]=rec_pos_mes_4[:]+ noise_r_3[:]
spos1=[]
spos2=[]
spos=np.zeros(shape =(352,2))
t_avg_4 =np.mean(t_Rec_real_4)
rad_domain = np.sqrt(x**2 + y**2) # radius from the origin of each point in domain
r_avg = t_avg*c
n=0
t=0
for m in range(8):
    for l in range (m,9):
        for k in range(l,9):
                    if(k!=m or k!=l):
                        for i in range(1000):
                             for j in range(1000):
                                if rad_domain[i,j] > r_avg  - 50  and  rad_domain[i,j] < r_avg  + 50:
                                    error[i,j] = src_Loc(rec_pos=np.array([rec_pos_modified_4[m],rec_pos_modified_4[l],rec_pos_modified_4[k]]),t_rec=np.array([t_Rec_real_4[m],t_Rec_real_4[l],t_Rec_real_4[k]]),c=c,src_pos=np.array([x[i,j],y[i,j]]))
                                else:
                                    error[i,j] = np.max(error)+1e8    
                        n+=1
                        t+=1
                        print(t)
                        spos[n-1,0]=x[np.where(error == np.min(error))][0]
                        spos[n-1,1]=y[np.where(error == np.min(error))][0]
                        plt.figure(figsize=(8,6))
                        plt.contourf(x,y,error,levels=50)
                        plt.plot(rec_pos_mes_4[:,0],rec_pos_mes_4[:,1],'b.',label='Receivers')
                        plt.plot(rec_pos_mes_4[:,0],rec_pos_mes_4[:,1],'g.',label='Receivers')
                        plt.plot(spos[:,0],spos[:,1],'r.',label='Source position estimated_2')
                        plt.plot(400,400,'r^',label='Source position Actual')
                        plt.plot(x[np.where(error == np.min(error))],y[np.where(error == np.min(error))],'g^',label='Source position estimated')
                        plt.plot()
                        plt.legend()
                        plt.xlim(-500,500)
                        plt.ylim(-500,500)
                        plt.xlabel('X(m)')
                        plt.ylabel('Y(m)')
                        plt.axis('equal')
                        plt.colorbar()
                        plt.title('Error contour for receivers with error in both time of arrival and known position')
                        spos[:,0]=spos[:,0]-400
                        spos[:,1]=spos[:,1]-400
              